# TensorFlow GPU avec Azure ML

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

In [ ]:
import sys
sys.version

In [ ]:
import utils

In [ ]:
import datetime
maintenant = datetime.datetime.now()
print("Date du run = ", maintenant)

In [ ]:
import azureml.core
print("Version Azure ML =", azureml.core.VERSION)

## 1. Workspace

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

## 2. AML Compute GPU

Les tailles NC, NCv2 et NCv3 sont optimisées pour les algorithmes et les applications nécessitant beaucoup de ressources réseau et de calculs. 

En voici quelques exemples : les applications et les simulations CUDA et OpenCL, l’intelligence artificielle et l’apprentissage profond. Équipée du GPU Tesla V100 de NVIDIA, la série NCv3 est axée sur les charges de travail informatiques à hautes performances. 

- La série NC utilise le processeur Intel Xeon E5-2690 v3 2.60GHz v3 (Haswell) et les machines virtuelles de la série NCv2 et NCv3 sont dotées du processeur Intel Xeon E5-2690 v4 (Broadwell).

- ND et NDv2La série ND est destinée à l’exécution de scénarios d’apprentissage et d’inférence pour le Deep Learning. Elle utilise le GPU NVIDIA Tesla P40 et le processeur Intel Xeon E5-2690 v4 (Broadwell). La série NDv2 utilise le processeur Intel Xeon Platinum 8168 (Skylake).

- Les tailles NV et NVv3 sont optimisées et conçues pour la visualisation à distance, la diffusion en continu, les jeux, l’encodage et les scénarios de VDI utilisant des infrastructures comme OpenGL ou DirectX. Ces machines virtuelles reposent sur le GPU Tesla M60 de NVIDIA.

- Les tailles NVv4 sont optimisées et conçues pour l’infrastructure VDI et la visualisation à distance. Avec des GPU partitionnés, NVv4 offre la taille adaptée aux charges de travail nécessitant des ressources GPU plus petites. Ces machines virtuelles sont associées au GPU AMD Radeon Instinct MI25.

https://docs.microsoft.com/fr-fr/azure/virtual-machines/windows/sizes-gpu

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

AmlCompute.supported_vmsizes(workspace = ws)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "gpuclusterNC6"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           min_nodes=1,
                                                           max_nodes=8
                                                           )

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

In [ ]:
liste = ws.compute_targets
for liste in liste:
    print("Ressources Compute du workspace :", liste)

In [ ]:
compute_target.get_status().serialize()

In [ ]:
# Statut du compute server
compute_target.list_nodes()

## 3. Données

In [ ]:
from azureml.core.dataset import Dataset
web_paths = ['http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path = web_paths)

In [ ]:
dataset = dataset.register(workspace = ws,
                           name = 'mnist dataset',
                           description='training and test dataset',
                           create_new_version=True)

In [ ]:
dataset.to_path()

## 4. Création projet et expérimentation

In [ ]:
import os
script_folder = './tf-resume-training'
os.makedirs(script_folder, exist_ok=True)

In [ ]:
from azureml.core import Experiment

experiment_name = 'Exemple11-TensorFlow'
experiment = Experiment(ws, name=experiment_name)

## 5. Création et exécution estimator TensorFlow

### Visualisation du code python :

In [ ]:
with open(os.path.join(script_folder, './tf_mnist_with_checkpoint.py'), 'r') as f:
    print(f.read())

In [ ]:
from azureml.train.dnn import TensorFlow

script_params={
    '--data-folder': dataset.as_named_input('mnist').as_mount()
}

estimator= TensorFlow(source_directory=script_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='tf_mnist_with_checkpoint.py',
                      use_gpu=True,
                      pip_packages=['azureml-dataprep[pandas,fuse]'])

### Exécution du Run

In [ ]:
tags = {"Type": "test" , "Langage" : "Python" , "Framework"  : "TensorFlow"}

In [ ]:
run = experiment.submit(estimator, tags=tags)
print(run)

### Widget pour suivi de l'exécution du run

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
run.get_details()

### Capture d'écran des résultats du run :

<img src="https://github.com/retkowsky/images/blob/master/GPUTF.jpg?raw=true">

### Liste des métriques du run

In [ ]:
run.get_metrics()

### Visualisation des métriques dans l'expérimentation depuis Azure ML Studio

In [ ]:
experiment

In [ ]:
# Statut du compute server
compute_target.list_nodes()

### Suppression compute instance

In [ ]:
liste = ws.compute_targets
for liste in liste:
    print("Ressources Compute du workspace :", liste)

In [ ]:
# Suppression du cluster
#compute_target.delete()

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, "(" , ct.type, ") :", ct.provisioning_state)

<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">